In [ ]:
import wandb
wandb.login()

In [ ]:
import pandas as pd 
import wandb
import numpy as np
import json

api = wandb.Api(timeout=50)
entity, project = "lm-informants", "0901_atr"  # set to your entity and project 
runs = api.runs(entity + "/" + project) 

In [ ]:
from tqdm import tqdm

summary_list, config_list, name_list = [], [], []
mega_df = pd.DataFrame()

all_data = []

for run_idx, run in tqdm(enumerate(runs), total=len(runs)):
    
    # add all the keys that are logged that you want to download
    keys = [
        "step",
        "auc",
        'strategy_used_is_train',
        ]
    
    end_keys = ['end_stats/mean_auauc']
                    
    if 'Model Eval Logs' in run.summary:
        model_eval_logs_exists = True
    else:
        model_eval_logs_exists = False
    
    # TODO: comment out, hackily setting to not download
    model_eval_logs_exists=False
#     print(model_eval_logs_exists)
    
    # download Model Eval Logs table
    if model_eval_logs_exists:
        
        path = run.summary['Model Eval Logs']['path']
        run.file(path).download(replace=True)

        table = json.load(open(path))
        
        model_eval_logs = pd.DataFrame(data=table['data'], columns=table['columns'])
#         display(df)
#         table = run.use_artifact(f"run-{run.id}-Model Eval Logs").get("Model Eval Logs")

#         model_eval_logs = run.summary['Model Eval Logs']
#         model_eval_logs = (run.history(keys=['Model Eval Logs'])['Model Eval Logs'][0])
#         print(model_eval_logs)
    
    history = run.scan_history()
    
    history_df = pd.DataFrame(history)
                
    # filter ones that were killed
    if run.state != "finished":
        print("filtering run (not finished): ", run.path)
        continue
        
    # get the experiment config
    config = {k: v for k,v in run.config.items()
         if not k.startswith('_')}
    
    exp = {}
    
    # tells us whether strategy_used_is_train_exists was logged as a metric
    strategy_used_is_train_exists = ('strategy_used_is_train' in history_df.columns)
    
    # if key doesn't exist, set to nan (though this shouldn't happen after filtering empty runs)
    exp.update({f"{col}": history_df[~history_df[col].isnull()][col].values if col in history_df.columns else np.nan for col in keys})
    
    # end stats should only be logged once
    end_key_results = {}
    for k in end_keys:
        val = history_df[~history_df[k].isnull()][k].values
        assert len(val) == 1
        end_key_results.update({f"{k}": val[0]})
    
    num_steps = len(exp[keys[0]])
    for k in keys:
        if isinstance(exp[k], float) and np.isnan(exp[k]):
            print(f'Warning: key {k} is nan for run: ', run.path)
            # set to list of nans
            exp[k] = [np.nan] * num_steps
            continue
        # only make sure that the non-nan keys have num_steps many entries
        assert len(exp[k]) == num_steps
        
    # convert dict of lists to list of dicts
    results = [dict(zip(exp,t)) for t in zip(*exp.values())]
    
#     print(results)
    
    for r in results:
        r.update({f"config/{key}": val for key, val in config.items()})
        r.update({f"end_results/{key}": val for key, val in end_key_results.items()})
        r.update({f"strategy": run.name})
        r.update({f"wandb_id": run.path})
        r.update({'tags': run.tags})
        
        if model_eval_logs_exists:
            table_row = model_eval_logs.iloc[int(r['step'])]
            strategy_used = table_row['strategy_for_this_candidate']
            r.update({'strategy_used': strategy_used})
            
            if strategy_used_is_train_exists:
                assert r['strategy_used_is_train'] == (strategy_used == 'train'), (f"logged strategy_used_is_train is {r['strategy_used_is_train']}"
                                                                              f" but strategy_used from table is {strategy_used}"
                                                                              f" for step {r['step']}")
            
            # if strategy_used_is_train wasn't logged, set it with the table logged at the end
            else:
                r['strategy_used_is_train'] = (strategy_used == 'train')
        
        # strategy_used_is_train will be nan if BOTH strategy_used_is_train wasn't directly logged AND Model Eval Logs wasn't logged (it defaults to nan if not in the columns when looping through keys)
        # strategy_used will be nan if model_eval_logs_exists = False (i.e. the Model Eval Logs table wasn't logged)
        else:
            r['strategy_used'] = np.nan
        
        
            
                
    all_data.extend(results)
    
#     print(all_data)
    
    
# get status
    
mega_df = pd.DataFrame(all_data)


In [ ]:
out_path = './0901_atr_0913.csv'
mega_df.to_csv(out_path)

In [39]:
mega_df = pd.read_csv('0901_atr.csv')

In [35]:
results_by_params = {}

temp = mega_df
for strat in mega_df['strategy'].unique():
    strat_df = mega_df[mega_df['strategy']==strat] 
#     print("strategy:", strat)
    for log in mega_df['config/log_log_alpha_ratio'].unique():
#         print("log:", log)
        log_df = strat_df[strat_df['config/log_log_alpha_ratio']==log]
        for prior in mega_df['config/prior_prob'].unique():
#             print("prior:", prior)
            prior_df = log_df[log_df['config/prior_prob']==prior]
#             display(temp)
            for max_updates in mega_df['config/max_updates_observe'].unique():
#                 display(len(temp))
#                 print('max updates:', max_updates)
#                 display(temp)
                observe_df = prior_df[prior_df['config/max_updates_observe']==max_updates]
                propose_df = observe_df[observe_df['config/max_updates_propose']==max_updates]
                
                key = (strat, log, prior, max_updates)
                num_exps = len(propose_df)
                mean_auauc = 
                results_by_param[]
                

0

0

0

3

0

...

6

0

9

0

7

In [36]:
temp2 = mega_df[mega_df['strategy']=='kl_train_model']
temp2 = temp2[temp2['config/log_log_alpha_ratio']==0.5]
temp2 = temp2[temp2['config/prior_prob']==0.35]
display(temp2)

,step,auc,strategy_used_is_train,end_stats/mean_auauc,config/run,config/n_init,config/strategy,config/tolerance,config/prior_prob,config/warm_start,...,config/pool_prop_edits,config/train_expect_type,config/log_log_alpha_ratio,config/max_updates_observe,config/max_updates_propose,config/metric_expect_assume_labels,strategy,wandb_id,tags,strategy_used
0,0.0,0.752471,0.0,0.923859,2,0,kl_train_model,0.000002,0.35,False,...,0,proposal_samples,0.5,NaN,NaN,False,kl_train_model,"[lm-informants, 0901_atr, se3y74u5]",[0901_test_atr],NaN
12,0.0,0.752471,0.0,0.842642,1,0,kl_train_model,0.000002,0.35,False,...,0,proposal_samples,0.5,NaN,NaN,False,kl_train_model,"[lm-informants, 0901_atr, v6ngl208]",[0901_test_atr],NaN
24,0.0,0.752471,0.0,0.926093,0,0,kl_train_model,0.000002,0.35,False,...,0,proposal_samples,0.5,NaN,NaN,False,kl_train_model,"[lm-informants, 0901_atr, ra95mhzc]",[0901_test_atr],NaN


In [40]:
mega_df['step'].unique()

array([0.])

In [29]:
display(len(mega_df[mega_df['config/prior_prob']==0.05]))

368